## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
ipDict = {
    '芜湖': '192.168.0.91',
    '北京': '100.72.64.19',
    '内蒙': '192.168.1.174',
    '泉州': '10.55.146.88',
    '南京': '172.181.217.43',
    '佛山': '192.168.126.12',
}

def ping_threading(ipDict):
    import threading
    import subprocess
    import time
    from queue import Queue


    # 定义工作线程
    WORD_THREAD = 10

    # 将需要 ping 的 ip 加入队列
    IP_QUEUE = Queue() 
    for i in ipDict:
        IP_QUEUE.put({'ip':ipDict[i],'region':i})

    # 定义一个执行 ping 的函数


    def ping_ip():
        while (not IP_QUEUE.empty()):
            cb = IP_QUEUE.get()
            test_ip = cb['ip']
            test_region = cb['region']
            res = subprocess.call('ping -c 1 -w 1 %s' % test_ip,stdout=subprocess.PIPE,shell=True)  # linux 系统将 '-n' 替换成 '-c' ，而且设置shell=True，详情请参考：https://xilou.info/p/118
            # 打印运行结果
            if res == 0:
                global region,ip
                region = test_region
                ip = test_ip
                # print(region,ip,True if res == 0 else False)
                break
        
    threads = []
    start_time = time.time()
    for i in range(WORD_THREAD):
        thread = threading.Thread(target=ping_ip)
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
    
    print('Ping程序运行耗时：%s' % (time.time() - start_time))
    return {'region':region,'ip':ip}

def getProxyURL(ip):
    return f'http://{ip}:12798'

def autoRegion():
    global region, proxy, proxyURL
    cb=ping_threading(ipDict)
    region = cb['region']
    ip = cb['ip']
    proxy = makeCLI(ip)
    proxyURL = getProxyURL(ip)
    # for key in ipDict:
    #     ip = ipDict[key]
    #     if (os.system(f'ping -c 1 -w 1 {ip}') == 0):
    #         # print('OK')
    #         global region, proxy
    #         region = key
    #         proxy = makeCLI(ip)
    #         break
    #     else:
    #         # print('Connection failed')
    #         continue

def makeCLI(ip):
    return f'export http_proxy={getProxyURL(ip)} && export https_proxy={getProxyURL(ip)}'

def setProxy():

    import os
    import subprocess
    from ipywidgets import interact, widgets
    from IPython.display import display, clear_output
    
    global region, proxy ,proxyURL,ip
    btnArray = []
    region = '未知'
    proxy = 'cd ./'  # 一句没有实际作用的命令作为占位
    ip=''

    def printSuccess(region):
        # print(f'已挂载【{region}】对应的代理')
        picked.button_style = 'info'
        picked.description = f'在使用【{region}】代理'
        # picked.icon = 'check'


    def printFail():
        # print(f'已挂载【{region}】对应的代理')
        picked.button_style = 'warning'
        picked.description = f'没有合适的代理'
        picked.icon = 'exclamation-triangle'


    def autoClick(region):
        for o in btnArray:  # 取消所有按钮被选中的样式
            o.button_style = ''
            o.icon = ''
        printSuccess(region)
        for k in btnArray:
            if k.description == region:
                k.button_style = 'success'
                k.icon = 'check'


    def btn_eventhandle(obj):
        global region, proxy
        for o in btnArray:  # 取消所有按钮被选中的样式
            o.button_style = ''
            o.icon = ''
        obj.button_style = 'success'
        obj.icon = 'check'
        region = obj.description
        ip = ipDict[region]
        proxy = makeCLI(ip)
        printSuccess(obj.description)


    picked = widgets.Button(
        value=False,
        description=f'自动侦测中...',
        disabled=False,
        button_style='warning',  # 'success', 'info', 'warning', 'danger' or ''
        tooltip='请点击下方的区名',
        icon='question-circle'  # (FontAwesome names without the `fa-` prefix)
    )

    for i in ipDict:
        btn = widgets.Button(
            description=i,
            disabled=False,
            button_style='',  # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click me',
            # icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        btn.on_click(btn_eventhandle)
        btnArray.append(btn)

    btnArray.insert(0, picked)

    for e in btnArray:
        display(e)

    autoRegion()
    clear_output(wait=True)

    if region != '未知':
        autoClick(region)
    else:
        clear_output(wait=True)
        printFail()

    for e in btnArray:
        display(e)
        
    return({'region':region,'proxy':proxy,'proxyURL':proxyURL})


cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-download"></i> 安装或更新

安装或更新OneClick-stale-diffusion本体

In [ ]:
import ipywidgets as widgets
from subprocess import getoutput
from IPython.display import display,clear_output
import os
import sys
# sys.path.append('../')
# from func.env import installDir
installDir = '/root/OneClick-stable-diffusion/'

btn = widgets.Button(
    value=False,
    description='正在安装中...',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    # tooltip='Description',
    # icon='check' # (FontAwesome names without the `fa-` prefix)
)

reinstallBtn = widgets.Button(
    value=False,
    description='需要重装？',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    # tooltip='Description',
    icon='question-circle' # (FontAwesome names without the `fa-` prefix)
)

def autoUpdate(obj):
    obj.description='正在自动更新...'
    obj.button_style='info'
    obj.icon='upload'
    stdout = !{proxy} &&\
    cd $installDir &&\
    git pull
    if 'Already up to date.' in stdout:
        obj.description='自动更新完成'
        obj.button_style='success'
        obj.icon='check-circle'

def reinstall(obj):
    obj.close()
    btn.description="正在重装..."
    btn.button_style='warning'
    btn.icon='exclamation-triangle'
    !rm -rf $installDir
    install_language_pack()

def install_language_pack():
    # print("开始安装")
    stdout =!{proxy} &&\
            cd /root &&\
            git clone https://github.com/lcolok/OneClick-stable-diffusion

    def getdirsize(dir):
       size = 0
       for root, dirs, files in os.walk(dir):
          size += sum([os.path.getsize(os.path.join(root, name)) for name in files])
       return size

    sz = getdirsize(installDir)
    # print(stdout)
    # print(sz)
    for i in stdout:
        # print(i)
        if sz>0 :
            if "already exists and is not an empty directory" in i:
                btn.description='已经安装过'
                btn.button_style='info'
                btn.icon='check-circle'
                autoUpdate(btn)
                display(reinstallBtn)
            else:
                btn.description='安装成功!'
                btn.button_style='success'
                btn.icon='check'
        else:
            btn.description="安装失败"
            btn.button_style='danger'
            btn.icon='exclamation-triangle'
  
            
reinstallBtn.on_click(reinstall)

display(btn)
install_language_pack()